In [1]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils import plot_model

import pandas as pd
import matplotlib.pyplot as plt
import pickle

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-12-04 22:00:26.476087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def get_data(filename):
    with open(filename) as training_file:
        csv_reader = csv.reader(training_file, delimiter=',')
        first_line = True
        temp_images = []
        temp_labels = []
        for row in csv_reader:
            if first_line:
                first_line = False
            else:
                temp_labels.append(row[0])
                image_data = row[1:785]
                image_data_as_array = np.array_split(image_data, 28)
                temp_images.append(image_data_as_array)
        images = np.array(temp_images).astype('float')
        labels = np.array(temp_labels).astype('float')
    return images, labels

In [3]:
train_path = './data/sign_mnist_train/sign_mnist_train.csv'
test_path = './data/sign_mnist_test/sign_mnist_test.csv'

training_images, training_labels = get_data(train_path)
testing_images, testing_labels = get_data(test_path)

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

(27455, 28, 28)
(27455,)
(7172, 28, 28)
(7172,)


In [4]:
# Data Augmentation
training_images = np.expand_dims(training_images, axis=3)
testing_images = np.expand_dims(testing_images, axis=3)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


validation_datagen = ImageDataGenerator(rescale=1. / 255)

print(training_images.shape)
print(testing_images.shape)


(27455, 28, 28, 1)
(7172, 28, 28, 1)


In [5]:
training_set = train_datagen.flow(
    training_images, training_labels,
    batch_size=32,
    shuffle=True
)

test_set = train_datagen.flow(
    testing_images, testing_labels,
    batch_size=32,
    shuffle=False
)

In [6]:
# CNN Model
classifier = Sequential()

# First layer
# classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(Convolution2D(32, (3, 3), input_shape = (28, 28, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Second layer
classifier.add(Convolution2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Third layer
classifier.add(Convolution2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
classifier.add(Flatten())

# Classification
classifier.add(Dense(256, activation='relu'))
classifier.add(Dense(26, activation='softmax'))

# Run the CNN model
# classifier.compile(loss='categorical_crossentropy',
classifier.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Display the model summary
classifier.summary()

# model = classifier.fit(
#         training_images,
#         steps_per_epoch=800,
#         epochs=25,
#         validation_data = testing_images,
#         validation_steps = 6500
#       )
print('Steps per epoch : ', len(training_images) / 32)
print('validation_steps : ', len(testing_images) / 32)

model = classifier.fit(
    training_set,
    steps_per_epoch=len(training_images) / 32,  # Adjust steps_per_epoch
    epochs=50,
    validation_data=test_set,
    validation_steps=len(testing_images) / 32  # Adjust validation_steps
)


classifier.save("classifier_CNN_MNIST_1.h5")

Model: "sequential"


2023-12-04 22:00:57.415818: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 128)        0         
 2D)      

2023-12-04 22:00:57.724032: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


858/857 [==============================] - ETA: 0s - loss: 2.8379 - accuracy: 0.1304

2023-12-04 22:02:31.131717: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


857/857 [==============================] - 98s 113ms/step - loss: 2.8379 - accuracy: 0.1304 - val_loss: 2.4323 - val_accuracy: 0.2359
Epoch 2/50
857/857 [==============================] - 86s 100ms/step - loss: 2.1022 - accuracy: 0.3243 - val_loss: 1.8307 - val_accuracy: 0.3977
Epoch 3/50
857/857 [==============================] - 93s 108ms/step - loss: 1.6860 - accuracy: 0.4488 - val_loss: 1.6138 - val_accuracy: 0.4681
Epoch 4/50
857/857 [==============================] - 90s 104ms/step - loss: 1.4435 - accuracy: 0.5254 - val_loss: 1.3852 - val_accuracy: 0.5287
Epoch 5/50
857/857 [==============================] - 89s 103ms/step - loss: 1.2645 - accuracy: 0.5800 - val_loss: 1.3312 - val_accuracy: 0.5485
Epoch 6/50
857/857 [==============================] - 83s 96ms/step - loss: 1.1157 - accuracy: 0.6294 - val_loss: 1.1607 - val_accuracy: 0.6097
Epoch 7/50
857/857 [==============================] - 79s 92ms/step - loss: 1.0195 - accuracy: 0.6594 - val_loss: 1.0849 - val_accuracy: 0.634

In [7]:
import json


In [8]:
# Save the history to a JSON file
with open('training_history.json', 'w') as f:
    json.dump(model.history, f)

# To load the history
with open('training_history.json', 'r') as f:
    loaded_history = json.load(f)

# Access loaded history
loaded_train_accuracy = loaded_history['accuracy']
loaded_test_accuracy = loaded_history['val_accuracy']